In [7]:
# Modeli eğit ve test et
train_model(model, train_loader, criterion, optimizer, num_epochs)


Epoch [1/5], Loss: 0.2673, Accuracy: 0.9379
Epoch [2/5], Loss: 0.0000, Accuracy: 1.0000
Epoch [3/5], Loss: 0.0000, Accuracy: 1.0000
Epoch [4/5], Loss: 0.0000, Accuracy: 1.0000
Epoch [5/5], Loss: 0.0000, Accuracy: 1.0000


NameError: name 'test_loader' is not defined

In [7]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset,random_split

from PIL import Image

In [2]:
batch_size = 64
learning_rate = 0.0001
num_epochs = 5
test_split_ratio = 0.2  # Verinin yüzde 20'sini test seti olarak ayır


In [3]:
# Özel Dataset sınıfı oluşturma
class SignatureDataset(Dataset):
    def __init__(self, real_dir, forg_dir, transform=None):
        self.real_dir = real_dir
        self.forg_dir = forg_dir
        self.transform = transform
        self.real_images = os.listdir(real_dir)
        self.forg_images = os.listdir(forg_dir)
        self.labels = [0] * len(self.real_images) + [1] * len(self.forg_images)  # 0: Gerçek, 1: Sahte

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if idx < len(self.real_images):
            img_path = os.path.join(self.real_dir, self.real_images[idx])
            label = 0
        else:
            img_path = os.path.join(self.forg_dir, self.forg_images[idx - len(self.real_images)])
            label = 1

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


In [8]:
# Veri dönüşümleri
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset oluşturma
full_dataset = SignatureDataset(real_dir="/Users/hbuzun/Documents/GitHub/Signature_recegnator/1_models/1_veri_setleri/1_Cedar_dataset/full_org",
                                forg_dir="/Users/hbuzun/Documents/GitHub/Signature_recegnator/1_models/1_veri_setleri/1_Cedar_dataset/full_forg",
                                transform=data_transforms)

# Eğitim ve test setlerine bölme
test_size = int(len(full_dataset) * test_split_ratio)
train_size = len(full_dataset) - test_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# DataLoader oluşturma
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [9]:
class SignatureDataset(Dataset):
    def __init__(self, real_dir, forg_dir, transform=None):
        self.real_dir = real_dir
        self.forg_dir = forg_dir
        self.transform = transform
        self.real_images = [img for img in os.listdir(real_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]
        self.forg_images = [img for img in os.listdir(forg_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.real_images) + len(self.forg_images)

    def __getitem__(self, idx):
        if idx < len(self.real_images):
            img_path = os.path.join(self.real_dir, self.real_images[idx])
            label = 0  # Orijinal imza
        else:
            img_path = os.path.join(self.forg_dir, self.forg_images[idx - len(self.real_images)])
            label = 1  # Sahte imza

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [10]:
# Modeli yükleme
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss ve optimizer tanımlama
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


/opt/anaconda3/envs/3-9-13_NN/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/3-9-13_NN/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/hbuzun/.cache/torch/hub/checkpoints/vgg16-397923af.pth
4.9%


KeyboardInterrupt: 

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

# VGG16 mimarisini katman katman oluşturma
class VGG16(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG16, self).__init__()
        
        # Özellik çıkarma (Feature extraction) katmanları
        self.features = nn.Sequential(
            # Blok 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Blok 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Blok 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Blok 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Blok 5
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Sınıflandırıcı (Classifier) katmanları
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Modeli tanımlama
model = VGG16(num_classes=2)

# Cihaz seçimi
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss fonksiyonu ve optimizer tanımlama
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Model özet bilgisi (isteğe bağlı)
print(model)


VGG16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

In [12]:
# Eğitim fonksiyonu
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        accuracy = correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

# Test fonksiyonu
def test_model(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    test_loss = running_loss / len(test_loader.dataset)
    accuracy = correct / total
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.4f}")


In [13]:
train_model(model, train_loader, criterion, optimizer, num_epochs)


KeyboardInterrupt: 

training: 68m 10.8s

In [19]:
test_model(model, test_loader, criterion)


Test Loss: 0.0000, Test Accuracy: 1.0000


In [20]:
# Modeli dosyaya kaydet
torch.save(model.state_dict(), 'signature_model.pth')
print("Model başarıyla kaydedildi.")


Model başarıyla kaydedildi.
